In [7]:
## Fainal code for data splitting as monthly_view and day_view - 2

import os
import pandas as pd
from datetime import datetime
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

def monthly_view(input_file, output_path):
    """
    Function to separate data into monthly sheets.
    :param input_file: Path to the CSV input file.
    :param output_path: Directory to save the output Excel file.
    """
    print("Starting the monthly view processing...")

    # Load the CSV file
    df = pd.read_csv(input_file)

    # Count the records in the input file
    total_records_input = len(df)
    print(f"Total records in input file: {total_records_input}")

    # Fix the date format
    def parse_date(value):
        try:
            return pd.to_datetime(value, format='%d/%m/%Y', errors='coerce')
        except Exception:
            return pd.NaT

    df['Slot booking date'] = df['Slot booking date'].apply(parse_date)
    df = df.dropna(subset=['Slot booking date'])

    # Add data updating timestamp column
    df['Data Updating Timestamp'] = datetime.now()

    # Group by month
    df['Month'] = df['Slot booking date'].dt.strftime('%B')
    df['Year'] = df['Slot booking date'].dt.year

    # Create the output file path
    output_file = os.path.join(output_path, "Monthly_Data.xlsx")

    records_transferred = 0

    for (year, month), group in df.groupby(['Year', 'Month']):
        sheet_name = f"{month}_{year}"

        # Sort the group by date
        group = group.sort_values(by='Slot booking date')
        group = group.drop(columns=['Month', 'Year'])

        if os.path.exists(output_file):
            book = load_workbook(output_file)
            if sheet_name in book.sheetnames:
                sheet = book[sheet_name]
                for row in dataframe_to_rows(group, index=False, header=False):
                    sheet.append(row)
                records_transferred += len(group)
            else:
                sheet = book.create_sheet(sheet_name)
                for row in dataframe_to_rows(group, index=False, header=True):
                    sheet.append(row)
                records_transferred += len(group)
            book.save(output_file)
        else:
            book = Workbook()
            del book['Sheet']
            sheet = book.create_sheet(sheet_name)
            for row in dataframe_to_rows(group, index=False, header=True):
                sheet.append(row)
            records_transferred += len(group)
            book.save(output_file)

    print(f"Monthly view processing completed. Total records transferred: {records_transferred}")
    print("Monthly view processing finished.\n")

def day_view(input_file, output_path):
    """
    Function to separate data into daily sheets per month.
    :param input_file: Path to the CSV input file.
    :param output_path: Directory to save the output Excel files.
    """
    print("Starting the daily view processing...")

    # Load the CSV file
    df = pd.read_csv(input_file)

    # Count the records in the input file
    total_records_input = len(df)
    print(f"Total records in input file: {total_records_input}")

    # Fix the date format
    def parse_date(value):
        try:
            return pd.to_datetime(value, format='%d/%m/%Y', errors='coerce')
        except Exception:
            return pd.NaT

    df['Slot booking date'] = df['Slot booking date'].apply(parse_date)
    df = df.dropna(subset=['Slot booking date'])

    # Add data updating timestamp column
    df['Data Updating Timestamp'] = datetime.now()

    # Extract year, month, and day
    df['Year'] = df['Slot booking date'].dt.year
    df['Month'] = df['Slot booking date'].dt.strftime('%B')
    df['Day'] = df['Slot booking date'].dt.day

    records_transferred = 0

    # Group by year and month
    for (year, month), month_group in df.groupby(['Year', 'Month']):
        month_folder = os.path.join(output_path, f"{month}_{year}")
        os.makedirs(month_folder, exist_ok=True)
        output_file = os.path.join(month_folder, f"{month}_{year}_Daily.xlsx")

        if os.path.exists(output_file):
            book = load_workbook(output_file)
        else:
            book = Workbook()
            del book['Sheet']

        for day, day_group in month_group.groupby('Day'):
            sheet_name = f"Day_{day}"

            # Sort the day group by date
            day_group = day_group.sort_values(by='Slot booking date')
            day_group = day_group.drop(columns=['Year', 'Month', 'Day'])

            if sheet_name in book.sheetnames:
                sheet = book[sheet_name]
                for row in dataframe_to_rows(day_group, index=False, header=False):
                    sheet.append(row)
                records_transferred += len(day_group)
            else:
                sheet = book.create_sheet(sheet_name)
                for row in dataframe_to_rows(day_group, index=False, header=True):
                    sheet.append(row)
                records_transferred += len(day_group)

        book.save(output_file)

    print(f"Daily view processing completed. Total records transferred: {records_transferred}")
    print("Daily view processing finished.\n")
    
    
import pandas as pd

def consolidate_data(input_file, output_file):
    """
    Function to consolidate all monthly sheets into a single sheet, sort the data, and remove duplicates.

    :param input_file: Path to the input Excel file containing monthly sheets.
    :param output_file: Path to save the consolidated Excel file.
    """
    print("Starting the data consolidation process...")

    try:
        # Load the Excel file using 'with' to ensure proper file closure
        with pd.ExcelFile(input_file) as excel_data:

            # Initialize an empty DataFrame to hold all the data
            consolidated_df = pd.DataFrame()

            # Iterate through each sheet and append the data
            for sheet_name in excel_data.sheet_names:
                sheet_data = pd.read_excel(input_file, sheet_name=sheet_name)
                consolidated_df = pd.concat([consolidated_df, sheet_data], ignore_index=True)

            # Remove duplicates
            consolidated_df = consolidated_df.drop_duplicates()

            # Sort the data by date (if the "Slot booking date" column exists)
            if "Slot booking date" in consolidated_df.columns:
                consolidated_df['Slot booking date'] = pd.to_datetime(
                    consolidated_df['Slot booking date'], errors='coerce'
                )
                consolidated_df = consolidated_df.sort_values(by='Slot booking date')

            # Save the consolidated data to a new Excel file
            consolidated_df.to_excel(output_file, index=False, sheet_name='Consolidated_Data')

        print(f"Data consolidation completed. File saved to: {output_file}")

    except Exception as e:
        print(f"An error occurred during data consolidation: {e}")


# Define the file paths
input_folder_path = 'C:/Users/ADMIN/Desktop/Automation/input'
monthly_output_path = "C:/Users/ADMIN/Desktop/Automation/output/monthly_view"
daily_output_path = "C:/Users/ADMIN/Desktop/Automation/output/day_view"
secondary_monthly_output_path = "C:/Automation/output/monthly_view"
secondary_daily_output_path = "C:/Automation/output/day_view"
monthly_excel_file = "C:/Automation/output/monthly_view/Monthly_Data.xlsx"
consolidated_output_file = "C:/Automation/output/monthly_view/Monthly_Consolidated_Data.xlsx"

# Check and get the first CSV file from the input folder
input_files = [file for file in os.listdir(input_folder_path) if file.endswith('.csv')]
if not input_files:
    print("No input files found in the input folder.")
    exit()

input_file_path = os.path.join(input_folder_path, input_files[0])

# Create directories if they don't exist
os.makedirs(monthly_output_path, exist_ok=True)
os.makedirs(daily_output_path, exist_ok=True)
os.makedirs(secondary_monthly_output_path, exist_ok=True)
os.makedirs(secondary_daily_output_path, exist_ok=True)

# Run the functions
monthly_view(input_file_path, monthly_output_path)
monthly_view(input_file_path, secondary_monthly_output_path)
day_view(input_file_path, daily_output_path)
day_view(input_file_path, secondary_daily_output_path)
consolidate_data(monthly_excel_file, consolidated_output_file)

Starting the monthly view processing...
Total records in input file: 12
Monthly view processing completed. Total records transferred: 12
Monthly view processing finished.

Starting the monthly view processing...
Total records in input file: 12
Monthly view processing completed. Total records transferred: 12
Monthly view processing finished.

Starting the daily view processing...
Total records in input file: 12
Daily view processing completed. Total records transferred: 12
Daily view processing finished.

Starting the daily view processing...
Total records in input file: 12
Daily view processing completed. Total records transferred: 12
Daily view processing finished.

Starting the data consolidation process...
Data consolidation completed. File saved to: C:/Automation/output/monthly_view/Monthly_Consolidated_Data.xlsx
